<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-Big-Data_Vision/blob/main/ML/1-tips_featuring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Target Variable : Tip

In [1]:
!cd

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('tips.csv')

In [ ]:
df.isnull().sum()

### Encoding

In [6]:
df['sex'].replace({'Female':0, 'Male':1}, inplace=True)

In [7]:
df["smoker"].replace({"No" : 0, "Yes" : 1}, inplace=True)
df["day"].replace({"Thur" : 0, "Fri" : 1, "Sat" : 2, "Sun" : 3}, inplace=True)
df["time"].replace({"Lunch" : 0, "Dinner" : 1}, inplace=True)

In [8]:
import numpy as np

In [9]:
df.replace('',np.nan,inplace=True) # '' -> np.nan

In [10]:
df['tip_percentage'] = (df['tip']/df['total_bill'])*100

In [11]:
df['tip_percentage'] = np.round(df['tip_percentage'],2)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
type(df)

pandas.core.frame.DataFrame

In [14]:
#import numpy as np
from sklearn.impute import SimpleImputer

In [15]:
data = df.values

In [16]:
y=df['tip'] # Target variable -> Tip

In [17]:
X=df.drop('tip',axis=1)

In [18]:
X.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_percentage'], dtype='object')

In [19]:
y.describe()

count    244.000000
mean       2.998279
std        1.383638
min        1.000000
25%        2.000000
50%        2.900000
75%        3.562500
max       10.000000
Name: tip, dtype: float64

In [20]:
data=X.values # Train data

In [21]:
y=y.values

In [ ]:
sum(np.isnan(data).flatten())

In [ ]:
sum(np.isnan(y).flatten())

In [24]:
imputer = SimpleImputer(strategy='median')

In [25]:
imputer.fit(data)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [26]:
# transform the dataset
data_trans = imputer.transform(data) # Imputed data set

### RFE

In [27]:
from sklearn.feature_selection import RFE

In [ ]:
#from sklearn.tree import DecisionTreeClassifier

In [28]:
from sklearn.svm import SVR

In [29]:
estimator = SVR(kernel="linear")

In [30]:
# define RFE
rfe = RFE(estimator, n_features_to_select=5)

In [31]:
# fit RFE
selector=rfe.fit(data_trans, y)

In [32]:
!cd

In [33]:
selector.support_

array([ True,  True,  True, False, False,  True,  True])

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_percentage'], dtype='object')

In [ ]:
# summarize all features
for i in range(data_trans.shape[1]):
  print('Column: %d,  Rank: %d' % (i,  rfe.ranking_[i]))

In [35]:
df1=pd.DataFrame(data_trans, columns=['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_percentage' ])

In [36]:
df1.columns

Index(['total_bill', 'sex', 'smoker', 'day', 'time', 'size', 'tip_percentage'], dtype='object')

In [ ]:
df1.drop([ 'time', 'size'], axis=1)

### PCA

In [38]:
from sklearn.decomposition import PCA

In [39]:
data_trans.shape

(244, 7)

In [40]:
# define the transform
trans = PCA(n_components=5)

In [ ]:
# https://stackoverflow.com/questions/55582413/put-results-of-pca-into-dataframe

In [41]:
# transform the data
X_dim = trans.fit_transform(data_trans)

In [42]:
# summarize data after the transform
print(X_dim[:3, :])

[[ -0.93290544 -10.455709    -1.45991492  -0.30022964  -0.37894007]
 [  8.76368653  -3.28016313  -1.66122008   0.99431705  -0.08947122]
 [ -0.99405246   1.01457123  -1.30498485   0.36913843  -0.29621359]]


In [43]:
trans.components_

array([[-0.93455023, -0.00759278, -0.0036134 , -0.01938127, -0.00844045,
        -0.0559023 ,  0.35067537],
       [ 0.34897766,  0.00099653,  0.00707371,  0.0247894 ,  0.00599564,
         0.02926261,  0.93629952],
       [ 0.03244857, -0.09797331,  0.02436618, -0.93737951, -0.32703167,
        -0.05343516,  0.01640814],
       [-0.05826144, -0.00965097, -0.19959764, -0.0400315 , -0.06265821,
         0.9752493 , -0.00578538],
       [-0.01899107,  0.02853285,  0.97609636, -0.0139226 ,  0.06873875,
         0.20272375, -0.00673377]])

In [44]:
df_pca = pd.DataFrame(X_dim)

In [45]:
df_pca.describe().round(2)

,0,1,2,3,4
count,244.00,244.00,244.00,244.00,244.00
mean,0.00,0.00,0.00,0.00,0.00
std,9.30,5.51,1.21,0.78,0.47
min,-28.75,-10.46,-1.67,-1.89,-0.85
25%,-4.81,-3.48,-1.03,-0.43,-0.39
50%,2.06,-0.64,-0.38,-0.16,-0.22
75%,6.17,2.64,1.49,0.32,0.50
max,30.98,47.10,2.54,3.12,1.00


In [46]:
df.describe()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
count,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,241.000000,244.000000
mean,19.785943,2.998279,0.643443,0.381148,1.725410,0.721311,2.556017,16.079754
std,8.902412,1.383638,0.479967,0.486667,1.155774,0.449276,0.947571,6.107020
min,3.070000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.560000
25%,13.347500,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,12.910000
50%,17.795000,2.900000,1.000000,0.000000,2.000000,1.000000,2.000000,15.475000
75%,24.127500,3.562500,1.000000,1.000000,3.000000,1.000000,3.000000,19.147500
max,50.810000,10.000000,1.000000,1.000000,3.000000,1.000000,6.000000,71.030000


### Regression Feature Selection

In [47]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [48]:
fs = SelectKBest(score_func=f_regression, k=5)

In [49]:
data_trans.shape

(244, 7)

In [50]:
y.shape

(244,)

In [51]:
# apply feature selection
X_selected = fs.fit_transform(data_trans, y)

In [52]:
parameters = fs.get_params(deep=True)

In [53]:
parameters

{'k': 5,
 'score_func': <function sklearn.feature_selection._univariate_selection.f_regression>}

In [54]:
parameters['score_func']

<function sklearn.feature_selection._univariate_selection.f_regression>

In [55]:
get_features = fs.fit(data_trans, y)

In [56]:
get_features

SelectKBest(k=5, score_func=<function f_regression at 0x7f3dc5ffd290>)

In [57]:
fs.get_support(indices=True)

array([0, 3, 4, 5, 6])

In [ ]:
print(X_selected.shape)

(244, 5)


In [ ]:
X_selected.shape[1]

5

In [ ]:
df_rfe_selected = pd.DataFrame(X_selected, columns=['total_bill', 'day', 'time', 'size', 'tip_percentage' ])

In [ ]:
df_rfe_selected.describe()

,total_bill,day,time,size,tip_percentage
count,244.000000,244.000000,244.000000,244.000000,244.000000
mean,19.785943,1.725410,0.721311,2.549180,16.079754
std,8.902412,1.155774,0.449276,0.943703,6.107020
min,3.070000,0.000000,0.000000,1.000000,3.560000
25%,13.347500,0.000000,0.000000,2.000000,12.910000
50%,17.795000,2.000000,1.000000,2.000000,15.475000
75%,24.127500,3.000000,1.000000,3.000000,19.147500
max,50.810000,3.000000,1.000000,6.000000,71.030000
